In [148]:
#get in the data
chinese = []
english_phoeneme = []
with open('dataset.txt') as f:
    for line in f:
        temp = line.split('\t')
        chinese.append(temp[1])
        english_phoeneme.append(temp[2])
english_phoeneme = [phoeneme.split('\n')[0].split() for phoeneme in english_phoeneme]
chinese = [list(word) for word in chinese]
for ele in chinese:
    if ' ' in ele:
        ele.remove(' ')
def prepare_vocab_input():
    vocab_inputs = []
    with open('./english_phoneme_vocabulary_output.txt') as file:
        for line in file:
            vocab_inputs.append(line.split('\n')[0])
    vocab_inputs.remove('_PAD')
    vocab_inputs.remove('_GO')
    vocab_inputs.remove('_EOS')
    vocab_inputs.remove('_UNK')
    #vocab_inputs = ['PAD', 'EOS'] + (vocab_inputs)
    vocab_inputs =  (vocab_inputs)+['PAD']
    return vocab_inputs
def prepare_vocab_predict():
    vocab_predict = list(chinese_id_dict.keys())
    return vocab_predict
# prepare chinese_id_dict
chinese_id_list = []
with open('./chinese_vocabulary.txt') as file7:
    for line in file7:
        chinese_id_list.append(line.split('\n')[0])
chinese_id_list =  chinese_id_list+['PAD']
chinese_id_dict = {}
for i in range(len(chinese_id_list)):
    chinese_id_dict[chinese_id_list[i]] = i
# finish preparation for chinese_id_dict
vocab_inputs = prepare_vocab_input()
vocab_predict = prepare_vocab_predict()
english_phoneme_dict = {}
for i in range(len(vocab_inputs)):
    english_phoneme_dict[vocab_inputs[i]] = i
for ele in chinese:
    if ' ' in ele:
        ele.remove(' ')

In [ ]:
english_phoneme_dict - char2numX
chinese_id_dict - char2numY

In [171]:
#prepare the english input
english_now = dict(zip(english_phoneme_dict.values(),english_phoneme_dict.keys()))
max_len = max([len(ele) for ele in english_phoeneme])
print(max_len)

english = [[english_phoneme_dict['PAD']]*(max_len - len(ele))+[english_phoneme_dict[phoneme] for phoneme in ele] for ele in english_phoeneme]
english = np.array(english)

75


In [177]:
#prepare the chinese output
chinese_now = dict(zip(chinese_id_dict.values(),chinese_id_dict.keys()))
max_len_chi = max([len(ele) for ele in chinese])
print(max_len_chi)

chinese = [[chinese_id_dict['PAD']]*(max_len_chi - len(ele))+[chinese_id_dict[char] for char in ele] for ele in chinese]
chinese = np.array(chinese)

39


In [181]:
def batch_data(x, y, batch_size):
    shuffle = np.random.permutation(len(x))
    start = 0
#     from IPython.core.debugger import Tracer; Tracer()()
    x = x[shuffle]
    y = y[shuffle]
    while start + batch_size <= len(x):
        yield x[start:start+batch_size], y[start:start+batch_size]
        start += batch_size

In [182]:
import numpy as np
import tensorflow as tf
import helpers
tf.reset_default_graph()
sess = tf.InteractiveSession()


In [185]:
x_seq_length = len(english[0])
y_seq_length = len(chinese[0])- 1

In [188]:
epochs = 2
batch_size = 128
nodes = 32
embed_size = 10

tf.reset_default_graph()
sess = tf.InteractiveSession()

# Tensor where we will feed the data into graph
inputs = tf.placeholder(tf.int32, (None, x_seq_length), 'inputs')
outputs = tf.placeholder(tf.int32, (None, None), 'output')
targets = tf.placeholder(tf.int32, (None, None), 'targets')

# Embedding layers
input_embedding = tf.Variable(tf.random_uniform((len(english_phoneme_dict), embed_size), -1.0, 1.0), name='enc_embedding')
output_embedding = tf.Variable(tf.random_uniform((len(chinese_id_dict), embed_size), -1.0, 1.0), name='dec_embedding')
date_input_embed = tf.nn.embedding_lookup(input_embedding, inputs)
date_output_embed = tf.nn.embedding_lookup(output_embedding, outputs)

with tf.variable_scope("encoding") as encoding_scope:
    lstm_enc = tf.contrib.rnn.BasicLSTMCell(nodes)
    _, last_state = tf.nn.dynamic_rnn(lstm_enc, inputs=date_input_embed, dtype=tf.float32)

with tf.variable_scope("decoding") as decoding_scope:
    lstm_dec = tf.contrib.rnn.BasicLSTMCell(nodes)
    dec_outputs, _ = tf.nn.dynamic_rnn(lstm_dec, inputs=date_output_embed, initial_state=last_state)
#connect outputs to 
logits = tf.contrib.layers.fully_connected(dec_outputs, num_outputs=len(chinese_id_dict), activation_fn=None) 
with tf.name_scope("optimization"):
    # Loss function
    loss = tf.contrib.seq2seq.sequence_loss(logits, targets, tf.ones([batch_size, y_seq_length]))
    # Optimizer
    optimizer = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

In [189]:
from sklearn.model_selection import train_test_split

In [190]:
X_train, X_test, y_train, y_test = train_test_split(english, chinese, test_size=0.33, random_state=42)

In [195]:
sess.run(tf.global_variables_initializer())
epochs = 10
for epoch_i in range(epochs):
    start_time = time.time()
    for batch_i, (source_batch, target_batch) in enumerate(batch_data(X_train, y_train, batch_size)):
        _, batch_loss, batch_logits = sess.run([optimizer, loss, logits],
            feed_dict = {inputs: source_batch,
             outputs: target_batch[:, :-1],
             targets: target_batch[:, 1:]})
    accuracy = np.mean(batch_logits.argmax(axis=-1) == target_batch[:,1:])
    print('Epoch {:3} Loss: {:>6.3f} Accuracy: {:>6.4f} Epoch duration: {:>6.3f}s'.format(epoch_i, batch_loss, 
                                                                      accuracy, time.time() - start_time))

Epoch   0 Loss:  0.905 Accuracy: 0.8427 Epoch duration: 73.806s
Epoch   1 Loss:  0.862 Accuracy: 0.8427 Epoch duration: 74.474s
Epoch   2 Loss:  0.786 Accuracy: 0.8563 Epoch duration: 73.188s
Epoch   3 Loss:  0.786 Accuracy: 0.8481 Epoch duration: 72.964s
Epoch   4 Loss:  0.757 Accuracy: 0.8555 Epoch duration: 73.219s
Epoch   5 Loss:  0.746 Accuracy: 0.8627 Epoch duration: 74.112s
Epoch   6 Loss:  0.744 Accuracy: 0.8596 Epoch duration: 73.939s
Epoch   7 Loss:  0.734 Accuracy: 0.8649 Epoch duration: 72.941s
Epoch   8 Loss:  0.741 Accuracy: 0.8618 Epoch duration: 72.730s
Epoch   9 Loss:  0.755 Accuracy: 0.8567 Epoch duration: 73.212s


In [226]:
source_batch, target_batch = next(batch_data(X_test, y_test, batch_size))

dec_input = np.zeros((len(source_batch), 1)) + chinese_id_dict['PAD']
for i in range(y_seq_length):
    batch_logits = sess.run(logits,
                feed_dict = {inputs: source_batch,
                 outputs: dec_input})
    prediction = batch_logits[:,-1].argmax(axis=-1)
    dec_input = np.hstack([dec_input, prediction[:,None]])
    
print('Accuracy on test set is: {:>6.3f}'.format(np.mean(dec_input == target_batch)))

Accuracy on test set is:  0.853


In [231]:
num_preds = 10
source_chars = [[english_now[l] for l in sent if english_now[l]!="PAD"] for sent in source_batch[:num_preds]]
dest_chars = [[chinese_now[l] for l in sent] for sent in dec_input[:num_preds, 1:]]

for date_in, date_out in zip(source_chars, dest_chars):
    print(''.join(date_in)+' => '+''.join(date_out))

WIHLYAHMNAEGWAHBSAYFAYN => PADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
RIHKAARDOWROWHHAAS => PADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
AEMBAHLAHPEYSOW => PADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD阿尔特拉斯
HHOWZEYKOWDAHRCHPLAENAHZ => PADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
VAEKLAAVHHAORAHCHEHK => PADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
MOWHHAAMEHDAALIYAEBTAHHHIY => PADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
WIHLYAHMJHEYMZPEHRIY => PADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPADPAD
MEHRIYERUHRK => PADPADPADPADPAD